In [17]:
# Magics
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [18]:
# Import libraries
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch import optim
from torchvision import models, transforms

In [19]:
# Import pre-trained model 
vgg = models.vgg19(pretrained=True).features

# Freeze the layers of model
for layer in vgg.parameters():
    layer.requires_grad_(False)
    
# Move the model over to GPU
GPU = 'cuda'
vgg.to(GPU)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace)
  (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (17): ReLU(inplace)
  (18): MaxPool2d(kernel_size=2, stride=2, padding=0, 

In [28]:
def load_image_as_tensor(img_path, max_size=400, shape=None):
    '''
    Function to load in images and convert them into provided size in x-y dimensions.
    The function will also convert the images to their respective tensors.
    
    img_path: String, path to the input image
    max_size: Int, Max. size allowed for the image while loading in the image (Default:400)
    shape: Int/Tuple, If provided will overide the image size parameter
    '''
    # Load in the image in RGB format
    img = Image.open(img_path).convert('RGB')
    
    # Set the size of the image to be processed
    if max(img.size) > max_size: # Large images slow down processing
        size = max_size
    else:
        size = max(img.size)
    
    # Override the max-size if the user provides a shape for the image 
    if shape is not None:
        size = shape
        
    # Define the transforms for converting image to a tensor
    transform = transforms.Compose([transforms.Resize(size),  # Resize
                                    transforms.ToTensor(), # Convert to tensor
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                                                         std=[0.229, 0.224, 0.225])]) # Normalize with imagenet stats
    image = transform(img)
    image = image[:3,:,:] # Select the RBG channels only if more channels available (alpha, etc.)
    image = image.unsqueeze(0) # Convert into a single batch of image with an added batch dimension
    
    return image